In [1]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.mixture import DPGMM

In [2]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file
from functions import *

In [3]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)



4771

In [4]:
num_upcoming_fights = len(df_upcoming)

In [5]:
df = df_upcoming.append(df)
len(df)

4784

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
def return_finish_type(winner, finish):
    #print(winner, finish)
    #Why overcomplicate things?  We can just use a few if statements
    if winner == 'Red':
        #print("HI")
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Red - DEC')
        if finish in ['SUB']:
            return('Red - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Red - KO/TKO')
    if winner == 'Blue':
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Blue - DEC')
        if finish in ['SUB']:
            return('Blue - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Blue - KO/TKO')
        
    #Test for NaN
    if finish != finish:
        return('')
    
    if finish == 'Overturned':
        return('')
    
    
    return ('error')

In [8]:
#This calls for the power of lambda!
df['finish_type'] = df.apply(lambda x: return_finish_type(x['Winner'], x['finish']), axis=1)

In [9]:
finish_list = ['Red - DEC', 'Red - SUB', 'Red - KO/TKO', 'Blue - DEC', 'Blue - SUB', 'Blue - KO/TKO']

#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.

for f in range(len(finish_list)):
    mask = df['finish_type'] == finish_list[f]
    df['label'][mask] = f
    
#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-af8f7542de83>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = f


In [10]:
df.rename(columns={'r_dec_odds': 'Red - DEC', 'r_sub_odds': 'Red - SUB', 'r_ko_odds': 'Red - KO/TKO',
                'b_dec_odds': 'Blue - DEC', 'b_sub_odds': 'Blue - SUB', 'b_ko_odds': 'Blue - KO/TKO'}, inplace=True)

In [11]:
df_label = df['label']
df_odds = df[finish_list]

In [12]:
df_train = df[num_upcoming_fights:]
odds_train = df_odds[num_upcoming_fights:]
label_train = df_label[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = df_odds[:num_upcoming_fights]
label_test = df_label[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

13
13
13
4771
4771
4771


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,R_rev_bout,B_rev_bout,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Ciryl Gane,Alexander Volkov,-155,125,64.516129,125.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,188.0,300.0,700.0,375.0,275.0,1600.0,Red - DEC,0.0
1,Tanner Boser,Ovince Saint Preux,-195,153,51.282051,153.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,175.0,250.0,1800.0,350.0,700.0,500.0,Red - KO/TKO,2.0
2,Raoni Barcelos,Timur Valiev,-230,180,43.478261,180.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,125.0,375.0,600.0,325.0,800.0,1250.0,Blue - DEC,3.0
3,Tim Means,Nicolas Dalby,-143,113,69.930070,113.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,165.0,400.0,800.0,225.0,600.0,1000.0,Red - DEC,0.0
4,Renato Moicano,Jai Herbert,-250,195,40.000000,195.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,240.0,550.0,175.0,550.0,325.0,1500.0,Red - SUB,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.0,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Blue - KO/TKO,5.0
4767,John Howard,Daniel Roberts,-210,175,47.619048,175.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4768,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4769,Mike Pierce,Julio Paulino,-420,335,23.809524,335.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - DEC,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,R_rev_bout,B_rev_bout,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Dustin Poirier,Conor McGregor,-134,105,74.626866,105.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,550.0,700.0,650.0,2000.0,180.0,160.0,,NaN
1,Gilbert Burns,Stephen Thompson,126,-157,126.000000,63.694268,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,350.0,150.0,500.0,2200.0,475.0,250.0,,NaN
2,Tai Tuivasa,Greg Hardy,-139,110,71.942446,110.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,350.0,350.0,1600.0,1800.0,165.0,250.0,,NaN
3,Irene Aldana,Yana Kunitskaya,-120,-105,83.333333,95.238095,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,160.0,162.0,1000.0,1200.0,425.0,900.0,,NaN
4,Sean O'Malley,Kris Moutinho,-900,525,11.111111,525.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,400.0,1200.0,800.0,1600.0,-275.0,850.0,,NaN
5,Carlos Condit,Max Griffin,150,-190,150.000000,52.631579,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,280.0,135.0,1000.0,1000.0,700.0,275.0,,NaN
6,Niko Price,Michel Pereira,130,-162,130.000000,61.728395,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,450.0,275.0,1400.0,600.0,285.0,250.0,,NaN
7,Ryan Hall,Ilia Topuria,195,-250,195.000000,40.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,500.0,225.0,475.0,1000.0,1000.0,150.0,,NaN
8,Trevin Giles,Dricus Du Plessis,-106,-118,94.339623,84.745763,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,210.0,450.0,900.0,525.0,450.0,275.0,,NaN
9,Jennifer Maia,Jessica Eye,-200,162,50.000000,162.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,105.0,250.0,550.0,1600.0,650.0,900.0,,NaN


In [15]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)

In [16]:
with open('../data/production_models_mov.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)

###REVERT THIS
#model_choice = 0

###


model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
1
2
3
4
5
6
7
YES
8
9
10
11
The model choice is 7
[['model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20', 'model_7_25_20', 'model_9_2_20', 'model_9_10_20', 'model_10_23_20', 'model_10_27_20', 'model_10_28_20', 'model_2021_06_24', 'model_2021_06_30'], ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()', 'GaussianNB(var_smoothing=1e-12)', 'GaussianNB(var_smoothing=1e-12)', 'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)', "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)", "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)", 'GradientBoostingClassifier(max_depth=95, min_samples_leaf=0.0015,\n             

In [17]:
def display_bets(df, min_ev, finish_list):
    for index, row in df.iterrows():
        print()
        print()
        print(f"{row['t1_name']} (RED) vs. {row['t2_name']} (BLUE)")
        for f in finish_list:
            ev = get_bet_ev(row[f], row[f+'_prob'])
            if ev > min_ev:
                print(f"BET: {f}. PROB: {row[f+'_prob']}.  EV: {ev}")

In [18]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, finish_list, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    #print(labels_train)
    print(len(df_train))
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    
    labels_train = labels_train.dropna()
    odds_train = odds_train[odds_train.index.isin(labels_train.index)]     
    df_train = df_train[df_train.index.isin(labels_train.index)] 
    
    
        
    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    prob_label_list = []
    for f in finish_list:
        prob_label_list.append(f + '_prob')
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #print(probs)
    #print()
    #print()
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, 0], odds_test[:, 1],
                            odds_test[:, 2], odds_test[:, 3],odds_test[:, 4], odds_test[:, 5],
                            probs[:, 0], probs[:, 1],probs[:, 2], probs[:, 3],probs[:, 4], probs[:, 5]))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name'] + finish_list + prob_label_list)

    display_bets(ev_prepped_df, input_ev, finish_list)
    
    return(ev_prepped_df)

In [19]:
output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, finish_list, verbose = True)

3728


Dustin Poirier (RED) vs. Conor McGregor (BLUE)
BET: Red - DEC. PROB: 0.9415973278889614.  EV: 512.038263127825


Gilbert Burns (RED) vs. Stephen Thompson (BLUE)
BET: Red - DEC. PROB: 0.995265540726228.  EV: 347.86949332680257


Tai Tuivasa (RED) vs. Greg Hardy (BLUE)
BET: Red - KO/TKO. PROB: 0.951098174335493.  EV: 152.04101619890565


Irene Aldana (RED) vs. Yana Kunitskaya (BLUE)
BET: Red - DEC. PROB: 0.8858069506697526.  EV: 130.30980717413567


Sean O'Malley (RED) vs. Kris Moutinho (BLUE)
BET: Blue - SUB. PROB: 0.24563209950443737.  EV: 317.5745691575436


Carlos Condit (RED) vs. Max Griffin (BLUE)
BET: Red - DEC. PROB: 0.8273961382618298.  EV: 214.41053253949534


Niko Price (RED) vs. Michel Pereira (BLUE)
BET: Red - DEC. PROB: 0.99393757808457.  EV: 446.66566794651345


Ryan Hall (RED) vs. Ilia Topuria (BLUE)
BET: Red - DEC. PROB: 0.8145734580222465.  EV: 388.7440748133479


Trevin Giles (RED) vs. Dricus Du Plessis (BLUE)
BET: Red - SUB. PROB: 0.21297050688682612.  EV: 112.

In [20]:
output_df

,t1_name,t2_name,Red - DEC,Red - SUB,Red - KO/TKO,Blue - DEC,Blue - SUB,Blue - KO/TKO,Red - DEC_prob,Red - SUB_prob,Red - KO/TKO_prob,Blue - DEC_prob,Blue - SUB_prob,Blue - KO/TKO_prob
0,Dustin Poirier,Conor McGregor,550.0,650.0,180.0,700.0,2000.0,160.0,0.941597,2.135903e-05,5.793205e-02,5.429823e-06,7.047070e-05,0.000373
1,Gilbert Burns,Stephen Thompson,350.0,500.0,475.0,150.0,2200.0,250.0,0.995266,6.299588e-06,9.278406e-07,4.698831e-03,5.524329e-08,0.000028
2,Tai Tuivasa,Greg Hardy,350.0,1600.0,165.0,350.0,1800.0,250.0,0.048892,1.045656e-08,9.510982e-01,3.136054e-07,3.739542e-07,0.000010
3,Irene Aldana,Yana Kunitskaya,160.0,1000.0,425.0,162.0,1200.0,900.0,0.885807,8.847089e-05,9.372625e-04,1.125100e-01,2.859651e-05,0.000629
4,Sean O'Malley,Kris Moutinho,400.0,800.0,-275.0,1200.0,1600.0,850.0,0.193079,4.094825e-05,5.605079e-01,5.288896e-04,2.456321e-01,0.000211
5,Carlos Condit,Max Griffin,280.0,1000.0,700.0,135.0,1000.0,275.0,0.827396,4.296445e-05,1.328036e-02,1.592277e-01,3.009346e-05,0.000023
6,Niko Price,Michel Pereira,450.0,1400.0,285.0,275.0,600.0,250.0,0.993938,7.211134e-07,1.541981e-03,2.837561e-03,1.450539e-04,0.001537
7,Ryan Hall,Ilia Topuria,500.0,475.0,1000.0,225.0,1000.0,150.0,0.814573,1.513532e-01,1.911899e-03,2.552640e-02,9.891230e-05,0.006536
8,Trevin Giles,Dricus Du Plessis,210.0,900.0,450.0,450.0,525.0,275.0,0.011632,2.129705e-01,7.546156e-01,5.748080e-03,1.558143e-06,0.015032
9,Jennifer Maia,Jessica Eye,105.0,550.0,650.0,250.0,1600.0,900.0,0.985141,1.036550e-03,4.327564e-05,1.334531e-02,1.283548e-04,0.000305
